In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error
from tifffile import tifffile
import numpy as np
import random
from tqdm.notebook import tqdm
from joblib import Parallel,delayed

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans,DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,StratifiedGroupKFold,StratifiedKFold,GroupKFold,KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor,StackingRegressor

In [2]:
label=pd.read_csv('train_answer.csv',header=None)
sample=pd.read_csv('sample_answer.csv',header=None)
label.columns=['image','location','AOD']
label['AOD']=np.log(label['AOD'])

In [3]:
def custom_scoring(true,pred):
    true=np.exp(true)
    pred=np.exp(pred)
    return (len(true)*sum(true*pred)-(sum(true)*sum(pred)))/np.sqrt(len(true)*sum(true**2)-sum(true)**2)/np.sqrt(len(true)*sum(pred**2)-sum(pred)**2)

In [4]:
class aerosol_vegetation:
    def __init__(self,data):
        self.data=data
        NDVI=(self.data[:,:,7]-self.data[:,:,3])/(self.data[:,:,7]+self.data[:,:,3])
        ARVI=(self.data[:,:,7]-(2*self.data[:,:,3]-self.data[:,:,1]))/(self.data[:,:,7]+(2*self.data[:,:,3]-self.data[:,:,1]))
        MSAVI=(2*self.data[:,:,7]+1-np.sqrt((2*self.data[:,:,7]+1)**2-8*(self.data[:,:,7]-self.data[:,:,3])))/2
        EVI=2.5*(self.data[:,:,7]-self.data[:,:,3])/(self.data[:,:,7]+6*self.data[:,:,3]-7.5*self.data[:,:,1]+1)
        AOT=self.data[:,:,1]/self.data[:,:,3]
        SR=self.data[:,:,7]/self.data[:,:,3]
        DSI=self.data[:,:,3]-self.data[:,:,1]
        # important 0,1,2,3,7

        self.data=np.concatenate([self.data[:,:,[0,1,2,3,7]],
                        NDVI.reshape((128,128,-1)),
                        ARVI.reshape((128,128,-1)),
                        MSAVI.reshape((128,128,-1)),
                        EVI.reshape((128,128,-1)),
                        AOT.reshape((128,128,-1)),
                        SR.reshape((128,128,-1)),
                        DSI.reshape((128,128,-1)),
                       ],axis=2)

In [5]:
def process_file(i,img_path,df):
    a=tifffile.imread(f'{img_path}/{df.loc[i][0]}')
    a=np.where(a>0.2,0.2,a)
    a=np.where(a<0.08,0.08,a)
    a=aerosol_vegetation(a).data
    a=np.concatenate([
                                a.max(axis=0).max(axis=0), # 0.91
                                a.max(axis=0).min(axis=0),
                                a.min(axis=0).min(axis=0),
                                a.min(axis=0).max(axis=0), # 0.9659
                                
                                a.min(axis=0).std(axis=0),
                                a.max(axis=0).std(axis=0),
                                a.std(axis=0).std(axis=0),
                                a.std(axis=0).max(axis=0),
                                a.std(axis=0).min(axis=0), # 0.9711
                                
                                a.mean(axis=0).min(axis=0),
                                a.mean(axis=0).max(axis=0),
                                a.mean(axis=0).std(axis=0),
        
                               ])
    return a


# Train Data Process

In [6]:
train_result_dfs = Parallel(n_jobs=-1)(delayed(process_file)(i,'train_images',label) for i in tqdm(range(len(label))))
train_result_dfs = np.array(train_result_dfs)

train=pd.DataFrame(train_result_dfs)
train.columns=[f'col_{i}' for i in range(1,train.shape[1]+1)]

train['target']=label.AOD

  0%|          | 0/4465 [00:00<?, ?it/s]

In [7]:
print(train.shape)

(4465, 145)


In [8]:
np.random.seed(42)
random.seed(42)
initial_centers = train.drop('target', axis=1).sample(n=15, random_state=42).values
clust=KMeans(n_clusters=15,init=initial_centers,random_state=42,n_init=1)
train['clust_1']=clust.fit_predict(train.drop('target',axis=1))
train.clust_1.value_counts()

9     507
5     507
3     480
13    480
4     394
8     382
2     371
7     324
10    251
14    237
6     182
1     142
0     127
11     60
12     21
Name: clust_1, dtype: int64

# Testing Data Process

In [9]:
test_result_dfs = Parallel(n_jobs=-1)(delayed(process_file)(i,'test_images',sample) for i in tqdm(range(len(sample))))
test_result_dfs = np.array(test_result_dfs)

test=pd.DataFrame(test_result_dfs)
test.columns=[f'col_{i}' for i in range(1,test.shape[1]+1)]

  0%|          | 0/1489 [00:00<?, ?it/s]

# Training and Inferencing model

In [10]:
gkf=GroupKFold(n_splits=10)

In [11]:
test_pred=[]
score=0
total=0
for i,(train_index,test_index) in enumerate(gkf.split(X=train,groups=train.clust_1)):
    train_data=train.loc[train_index]
    valid_data=train.loc[test_index]
    X_train,y_train=train_data.drop(['target','clust_1'],axis=1),train_data.target
    X_valid,y_valid=valid_data.drop(['target','clust_1'],axis=1),valid_data.target
    
    model=Pipeline(steps=[
    ('voting',VotingRegressor([
        ('cat',CatBoostRegressor(iterations=300,random_state=42,verbose=False)),
        ('lgbm',LGBMRegressor(random_state=42)),
        ('xgb',XGBRegressor(random_state=42)),
        ('rf',RandomForestRegressor(random_state=42))
    ])),
    ])
    
    model.fit(X_train,y_train)
    print(f"Kfold:{i+1}")
    print('train:',mean_absolute_error(y_train,model.predict(X_train)),mean_squared_error(y_train,model.predict(X_train)))
    print('valid:',mean_absolute_error(y_valid,model.predict(X_valid)),mean_squared_error(y_valid,model.predict(X_valid)))
    print('train:',custom_scoring(y_train,model.predict(X_train)))
    print('valid:',custom_scoring(y_valid,model.predict(X_valid)))
    score+=custom_scoring(y_valid,model.predict(X_valid))*len(y_valid)
    total+=len(y_valid)
    print('===================================================')
    test_pred.append(model.predict(test))
print("final_average_score:",score/total)

Kfold:1
train: 0.09574397165744013 0.01600289453052425
valid: 0.28440918484139116 0.14313592296486494
train: 0.9873379739303569
valid: 0.937532450947988
Kfold:2
train: 0.09532093677296204 0.01606129723891319
valid: 0.3171254881715636 0.1773147527211633
train: 0.9878760564513326
valid: 0.8745630038029015
Kfold:3
train: 0.09540472347369387 0.015952514258161073
valid: 0.29435366862467766 0.14559962461632292
train: 0.986705662915915
valid: 0.8844654357701538
Kfold:4
train: 0.09579009458011914 0.016258599373001354
valid: 0.26578138904612436 0.13664289608461871
train: 0.9869597975901424
valid: 0.8611424652225604
Kfold:5
train: 0.09395751928373602 0.015554832634086822
valid: 0.36160500142936086 0.24329621630539325
train: 0.9869783529739058
valid: 0.9113619743254857
Kfold:6
train: 0.09205729505857176 0.01454969129045549
valid: 0.5550208952555173 0.5987235237865861
train: 0.9917204178675696
valid: 0.6238311009810057
Kfold:7
train: 0.09626767901779694 0.016176083897220767
valid: 0.26201415738504

# Ensemble Prediction

In [23]:
final_prediction=np.array(test_pred)
sample.iloc[:,1]=final_prediction.mean(axis=0)
sample.iloc[:,1]=np.exp(final_prediction.mean(axis=0))
sample.to_csv("sub_10.csv",header=None,index=None)
sample.describe()

,1
count,1489.000000
mean,0.192573
std,0.196179
min,0.016972
25%,0.077456
50%,0.115618
75%,0.226770
max,1.344628
